In [8]:
from selenium import webdriver
import os
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
from datetime import datetime
import db_multiproc_calc_neutral2 as dmc
from random import *

rf_rate = 0.0475
k_do_download = True

cwd = os.getcwd()

executable_path = os.path.join(cwd,'chromedriver.exe')
display(executable_path)

date_str = datetime.now().strftime("%Y-%m-%d")
date_str = '2023-03-24'
file_date_str = date_str

download_path = os.path.join(cwd, 'local_download')
file_date_str

'C:\\Users\\mcbri\\PycharmProjects\\futureDataCapture\\chromedriver.exe'

'2023-03-24'

In [9]:
#############################
# get the Futures contracts and months we are needing and build a list of all the distinct contracts

df_future_month_def = pd.read_csv('futures_months.csv')

target_years = [23]

all_syms = []
for idx, row in df_future_month_def.iterrows():
    for y in target_years:
        for m in row['Month']:
            all_syms.append(row['symbol_root']+m+f'{y:.0f}')

In [10]:
def get_futures_contract_dates(file_str):
    df_fc = pd.read_csv('future_contract_spec.csv', converters={'':pd.eval})
    df_ctr_names = df_fc.iloc[:,1]
    df_ctr_names.name = 'futures_contract'
    df_ctr_dates = pd.DataFrame([[eval(x)[0] for x in df_fc.iloc[:,2]],
                 [eval(x)[1] for x in df_fc.iloc[:,2]]], index=['first_notice', 'expiration']).T
    df_ret = pd.concat([df_ctr_names, df_ctr_dates], axis=1)

    return df_ret

def get_option_contract_expiration(filename):
    dffc2 = pd.read_csv(filename, converters={'':pd.eval})
    opt_ctr = []
    for i, row in dffc2.iterrows():
        x = eval(row[2])
        for xx, yy in zip(x[2], x[3]):
            if not xx == '':
                opt_ctr.append([xx,pd.to_datetime(yy)])

    opt_ctr_exp = pd.DataFrame(
                    opt_ctr,
                    columns=['futures_contract', 'option_contract_expiration'])
    opt_str_exp = opt_ctr_exp.sort_values(by='option_contract_expiration')
    return opt_str_exp

In [11]:
df_1 = get_futures_contract_dates('future_contract_spec.csv')
df_2 = get_futures_contract_dates('future_contract_spec.2.csv')
df_fut = pd.concat([df_1, df_2])

In [12]:
df_1 = get_option_contract_expiration('future_contract_spec.csv')
df_2 = get_option_contract_expiration('future_contract_spec.2.csv')
df_opt_ctr = pd.concat([df_1, df_2])

In [13]:
df_opt_ctr.to_csv('option_contract_expiration.csv')

In [14]:
df_fut.to_csv('futures_contract_dates.csv')